In [1]:
import numpy as np
import TensorFrost as tf

tf.initialize(tf.opengl)


TensorFrost module loaded in debug mode! Expect slow performance.
n_body:
  Kernel count: 2
  Intermediate buffers: 1
  Host readbacks: 0
  Host writes: 0
  Lines of generated code: 418
  IR Compile time: 48.193298 ms
  Compiler time: 1291.021362 ms



RuntimeError: Input types float, int, are not valid for operation mul

In [30]:

dt = 0.0002
ParticleN = 10000
ImageW = 800
ImageH = 800

IC_Radius = 5.0

def Force(x1, x2):
    r = x2 - x1
    N = r.shape[0]
    dist = tf.norm(r)
    dist = tf.reshape(dist, [N, N, 1])
    return 0.2 * r / (dist ** 3.0 + 1e-3)

def n_body():
    X = tf.input([-1, 2], tf.float32)
    N = X.shape[0]
    V = tf.input([N, 2], tf.float32)

    i, j, k = tf.indices([N, N, 2])
    
    Fij = Force(X[i, k], X[j, k])
    Fij = tf.select(i == j, 0.0, Fij)
    
    Fi = tf.sum(Fij, axis=1)

    Vnew = V + Fi * dt
    Xnew = X + Vnew * dt

    return [Xnew, Vnew]

nbody = tf.compile(n_body)

def render_particles():
    X = tf.input([-1, 2], tf.float32)
    N = X.shape[0]
    atomic_canvas = tf.zeros([ImageW, ImageH], tf.int32)

    i, = tf.indices([N])
    xpos = (X[i, 0] / IC_Radius + 0.5) * float(ImageW)
    ypos = (X[i, 1] / IC_Radius + 0.5) * float(ImageW)
    is_inside = (xpos >= 0.0) & (xpos < float(ImageW)) & (ypos >= 0.0) & (ypos < float(ImageH))

    def drawPoint():
        xi = tf.int(xpos)
        yi = tf.int(ypos)
        xf = xpos - tf.float(xi)
        yf = ypos - tf.float(yi)
        w00 = (1.0 - xf) * (1.0 - yf)
        w01 = xf * (1.0 - yf)
        w10 = (1.0 - xf) * yf
        w11 = xf * yf

        tf.scatterAdd(atomic_canvas[xi, yi], tf.int(w00 * 255.0))
        tf.scatterAdd(atomic_canvas[xi + 1, yi], tf.int(w01 * 255.0))
        tf.scatterAdd(atomic_canvas[xi, yi + 1], tf.int(w10 * 255.0))
        tf.scatterAdd(atomic_canvas[xi + 1, yi + 1], tf.int(w11 * 255.0))

    tf.if_cond(is_inside, drawPoint)

    i, j, ch = tf.indices([ImageW, ImageH, 3])
    image = tf.float(atomic_canvas[i, j]) / 255.0
    return [image]

render = tf.compile(render_particles)

n_body:
  Kernel count: 2
  Intermediate buffers: 1
  Host readbacks: 0
  Host writes: 0
  Lines of generated code: 418
  IR Compile time: 50.460499 ms
  Compiler time: 1325.206177 ms

render_particles:
  Kernel count: 3
  Intermediate buffers: 1
  Host readbacks: 0
  Host writes: 0
  Lines of generated code: 415
  IR Compile time: 68.202499 ms
  Compiler time: 1280.944824 ms



In [31]:
import time

X = np.random.randn(ParticleN, 2).astype(np.float32)
#V = np.zeros([N, 2], np.float32)
#initialize with rotation
V = np.zeros([ParticleN, 2], np.float32)
V[:, 0] = -X[:, 1] * 5.1
V[:, 1] = X[:, 0] * 5.1

Xtf = tf.tensor(X)
Vtf = tf.tensor(V)

tf.show_window(ImageW, ImageH, "N-body simulation")

init_time = time.time()

while not tf.window_should_close():
    cur_time = time.time() - init_time
    time_tf = tf.tensor(np.array([cur_time], np.float32))
    Xtf, Vtf = nbody(Xtf, Vtf)
    frame_tf, = render(Xtf)
    tf.render_frame(frame_tf)
    render_time = time.time() - init_time - cur_time
    tf.imgui_text("Simulation time: %.3f ms" % (render_time * 1000))
    tf.imgui_text("Particles: %d" % ParticleN)
    tf.imgui_text("FPS: %.1f" % (1.0 / (render_time + 1e-6)))
    tf.imgui_text("Time step: %.3f" % dt)


tf.hide_window()